In [ ]:
import logging
import warnings
import random
from tqdm import tqdm
from collections import Counter

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

import os


import transformers
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import (
    BertTokenizer,
    BertModel,
    PretrainedConfig,
    PreTrainedModel,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)


from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_recall_fscore_support,
    accuracy_score,
    confusion_matrix
)

from huggingface_hub import HfApi, create_repo

warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.tokenization_utils_base")
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

torch.set_default_tensor_type(torch.FloatTensor)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

aspects = ['appearance', 'aroma', 'palate', 'taste']

def set_random_set(seed):
    transformers.set_seed(seed)  # For transformers
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)

set_random_set(2024)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
vocab_size = tokenizer.vocab_size
bert = BertModel.from_pretrained('bert-base-uncased')
hf_token = ""
del tokenizer, bert # only download for cached
torch.cuda.empty_cache()

2025-09-17 01:56:02.700772: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758074162.899337      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758074162.951631      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [2]:
class BeerReviewDataset(Dataset):
    def __init__(self, ids, absa_1, absa_2, attention_masks, token_type_ids, labels):
        self.ids = ids
        self.absa_1 = absa_1
        self.absa_2 = absa_2
        self.attention_masks = attention_masks
        self.token_type_ids = token_type_ids
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.ids[idx],
            'absa_1': self.absa_1[idx],
            'absa_2': self.absa_2[idx],
            'attention_mask': self.attention_masks[idx],
            'token_type_ids': self.token_type_ids[idx],
            'labels': self.labels[idx]
        }

In [3]:
class BERTConfig(PretrainedConfig):
    model_type = "bert_with_absa"

    def __init__(self, absa_method=None, num_classes=2, class_weight=None, **kwargs):
        super().__init__(**kwargs)
        self.absa_method = absa_method
        self.num_classes = num_classes
        self.class_weight = class_weight

class InnerBert(nn.Module):
    def __init__(self, num_classes):
        super(InnerBert, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(768, num_classes)
        

    def forward(self, inputs_embeds=None, attention_mask=None, token_type_ids=None):      
        _, pooled_output = self.bert(inputs_embeds=inputs_embeds, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=False)
        x = self.dropout(pooled_output)
        logits = self.fc(x)

        return logits

class BERTModel(PreTrainedModel):
    config_class = BERTConfig

    def __init__(self, config):
        super().__init__(config)
        self.num_classes = config.num_classes
        self.absa_method = config.absa_method
        
        self.bert_sent = InnerBert(self.num_classes)

        if self.absa_method:
            self.absa_fc = nn.Linear(1, 768)
            self.bert_absa = InnerBert(self.num_classes)

        if config.class_weight is not None:
            class_weight = torch.tensor(config.class_weight, dtype=torch.float32)
            self.criterion = nn.CrossEntropyLoss(weight=class_weight.to(device))
        else:
            self.criterion = nn.CrossEntropyLoss()

        self.init_weights()


    def forward(self, input_ids=None, absa_1=None, absa_2=None, attention_mask=None, token_type_ids=None, labels=None):
        inputs_embeds = self.bert_sent.bert.embeddings(input_ids=input_ids, token_type_ids=token_type_ids)
        logits_sent = self.bert_sent(inputs_embeds, attention_mask, token_type_ids)
        
        if self.absa_method:
            absa_1 = self.absa_fc(absa_1)
            absa_2 = self.absa_fc(absa_2)
            absa_concat = torch.cat((absa_1, absa_2), dim=1)
            token_type_ids_absa = torch.tensor([0, 1]).unsqueeze(0).repeat(absa_concat.shape[0], 1).to("cuda")
            
            logits_absa = self.bert_absa(absa_concat, None, token_type_ids_absa)
            
            logits_sent += logits_absa
        
        loss = None
        if labels is not None:
            loss = self.criterion(logits_sent, labels)
    
        return SequenceClassifierOutput(loss=loss, logits=logits_sent)

In [4]:
def train_and_eval_model(aspect, method, absa_method, model, train_dataset, val_dataset, test_dataset, val_null_count, batch_size, num_epochs, lr, weight_decay=0.01, patience=5):

    model_id = f"trungpq/rlcc-new-{aspect}-{method}-absa-{absa_method}"
    try:
        repo_info = HfApi().repo_info(repo_id=model_id, token=hf_token)
    except Exception as e:
        if "404" in str(e):  # If repo does not exist
            create_repo(model_id, token=hf_token)
        else:
            print(f"An error occurred: {e}")
    
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = torch.argmax(torch.tensor(logits), dim=1).numpy()
        labels = labels.flatten()
    
        # if val_null_count > 0:
        #     null_preds = np.full(val_null_count, 3)
        #     null_labels = np.full(val_null_count, 3)
        #     preds = np.concatenate([preds, null_preds])
        #     labels = np.concatenate([labels, null_labels])
    
        # --- macro ---
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
            labels, preds, average="macro"
        )
        # --- micro ---
        precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
            labels, preds, average="micro"
        )
    
        cm = confusion_matrix(labels, preds)
    
        tp = np.diag(cm)
        fp = cm.sum(axis=0) - tp
        fn = cm.sum(axis=1) - tp
        tn = cm.sum() - (tp + fp + fn)
    
        total_tp = tp.sum()
        total_fp = fp.sum()
        total_fn = fn.sum()
        total_tn = tn.sum()
    
        return {
            "accuracy": accuracy_score(labels, preds),
            "f1_macro": f1_macro,
            "precision_macro": precision_macro,
            "recall_macro": recall_macro,
            "f1_micro": f1_micro,
            "precision_micro": precision_micro,
            "recall_micro": recall_micro,
            "total_tf": [total_tp, total_fp, total_tn, total_fn],
        }


    num_warmup_steps = int(len(train_dataset) / batch_size)
    num_logging_steps = int(num_warmup_steps / 5)
    if num_logging_steps == 0:
        num_logging_steps = 1

    training_args = TrainingArguments(
        output_dir=f"./reviews/results/{aspect}/",
        eval_strategy="epoch",   
        save_strategy="epoch",
        learning_rate=lr,
        max_grad_norm=1.0,
        warmup_steps=num_warmup_steps,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=weight_decay,
        load_best_model_at_end=True,  # For early stopping
        metric_for_best_model="eval_f1_macro",
        greater_is_better=True,
        save_total_limit=1,  # Limit to save the best model only
        logging_dir='./logs',
        logging_steps=num_logging_steps,
        save_steps=10,
        report_to="none",
        disable_tqdm=False,
        push_to_hub=False,
        hub_model_id=model_id,
        hub_token=hf_token
    )

    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=patience,
        early_stopping_threshold=0.005  # Minimum change to qualify as an improvement
    )

    # Initialize Trainer with model, arguments, train and evaluation datasets, and metrics
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping_callback]
    )

    # Train the model
    trainer.train()
    trainer.push_to_hub()
    
    # tạo folder nếu chưa có
    os.makedirs("phase2_log", exist_ok=True)
    
    history = pd.DataFrame(trainer.state.log_history)
    history.to_csv(f"phase2_log/{aspect}_{method}_{absa_method}.csv", index=False)
    
    torch.cuda.empty_cache()

    trainer = Trainer(
        model=model,
        args=training_args,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

   # Evaluate the model
    torch.cuda.empty_cache()
    eval_results = trainer.evaluate()
    print(f"Test results: {eval_results}")

    # --- bổ sung labels/preds ---
    logits, labels = trainer.predict(test_dataset)[:2]
    preds = torch.argmax(torch.tensor(logits), dim=1).numpy()
    eval_results["eval_labels"] = labels.tolist()
    eval_results["eval_preds"] = preds.tolist()

    del trainer
    return eval_results

In [5]:
from datasets import load_dataset

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

def get_dataset(aspect, method, absa_method, tokenizer, set_name, device='cuda', upsampling_time=5):

    if not absa_method:
        absa_method = "min"
    
    df = load_dataset(f"trungpq/rlcc-new-data-{aspect}")[set_name].to_pandas()
    df = df[['sentences_1', f'absa_{absa_method}_1', 'sentences_2', f'absa_{absa_method}_2', aspect]]
    if set_name == "train":
        df.dropna(inplace=True)

    labels = np.where(
        df[aspect].isnull(),
        3,
        df[aspect].replace({-1: 0, 0: 1, 1: 2}).values
    )
    null_count = (labels == 3).sum().item()
    
    df.dropna(inplace=True)
    # df = df.sample(n=100)

    sent_1 = df['sentences_1'].astype(str).to_list()
    sent_2 = df['sentences_2'].astype(str).to_list()

    absa_1 = df[f'absa_{absa_method}_1'].astype(float).to_list()
    absa_2 = df[f'absa_{absa_method}_2'].astype(float).to_list()

    absa_1 = torch.tensor(absa_1, dtype=torch.float32).unsqueeze(1).unsqueeze(1)
    absa_2 = torch.tensor(absa_2, dtype=torch.float32).unsqueeze(1).unsqueeze(1)

    labels = torch.tensor(df[aspect].replace({-1: 0, 0: 1, 1: 2}).values, dtype=torch.long)
    
    if method == 'upsample_replacement' and set_name == 'train':
        class_0_indices = [i for i, label in enumerate(labels) if label == 0]
        class_1_indices = [i for i, label in enumerate(labels) if label == 1]
        class_2_indices = [i for i, label in enumerate(labels) if label == 2]

        num_max_class = max(len(class_0_indices), len(class_1_indices), len(class_2_indices))
        resampled_indices = []
        for class_indices in [class_0_indices, class_1_indices, class_2_indices]:
            if len(class_indices) < num_max_class:
                num_upsampling = num_max_class
                resampled_indices.append(np.random.choice(class_indices, num_upsampling, replace=True))
            else:
                resampled_indices.append(class_indices)

        upsampled_indices = np.concatenate(resampled_indices)
        np.random.shuffle(upsampled_indices)

        sent_1 = [sent_1[i] for i in upsampled_indices]
        sent_2 = [sent_2[i] for i in upsampled_indices]
        absa_1 = absa_1[upsampled_indices]
        absa_2 = absa_2[upsampled_indices]
        labels = labels[upsampled_indices]

    sent_1 = [sent.replace("\n", ",").replace(".", " ") for sent in sent_1]
    sent_2 = [sent.replace("\n", ",").replace(".", " ") for sent in sent_2]

    all_sentence_pairs = list(zip(sent_1, sent_2))

    max_seq_length = 128
    total_input_ids = []
    total_input_mask = []
    total_segment_ids = []

    for s1, s2 in tqdm(all_sentence_pairs):
        tokens_a = tokenizer.tokenize(s1)
        tokens_b = tokenizer.tokenize(s2)

        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3) #account for [CLS] [SEP] [SEP]

        tokens = ["[CLS]"] + tokens_a + ["[SEP]"] + tokens_b + ["[SEP]"]
        segment_ids = [0] * (len(tokens_a) + 2) + [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)

        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        total_input_ids.append(torch.tensor([input_ids], dtype=torch.long).to(device))
        total_input_mask.append(torch.tensor([input_mask], dtype=torch.long).to(device))
        total_segment_ids.append(torch.tensor([segment_ids], dtype=torch.long).to(device))

    assert len(total_input_ids) == len(all_sentence_pairs)
    assert len(total_input_mask) == len(all_sentence_pairs)
    assert len(total_segment_ids) == len(all_sentence_pairs)

    input_ids = torch.cat(total_input_ids, dim=0).to('cpu')
    attention_masks = torch.cat(total_input_mask, dim=0).to('cpu')
    token_type_ids = torch.cat(total_segment_ids, dim=0).to('cpu')

    labels = torch.tensor(labels, dtype=torch.long)

    return null_count, (input_ids, absa_1, absa_2, attention_masks, token_type_ids, labels)

In [6]:
def compute_class_weights(labels):
    class_counts = Counter(labels)
    total_count = sum(class_counts.values())
    class_weights = [total_count / class_counts[i] if i in class_counts else 1.0 for i in range(3)]
    return class_weights

def get_datataset_and_model(method, absa_method, aspect):

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    _, train_data = get_dataset(aspect, method, absa_method, tokenizer, 'train')
    
    null_count, val_data = get_dataset(aspect, method, absa_method, tokenizer, 'validation')

    null_count, test_data = get_dataset(aspect, method, absa_method, tokenizer, 'test')

    train_dataset = BeerReviewDataset(*train_data)

    val_dataset = BeerReviewDataset(*val_data)
    
    test_dataset = BeerReviewDataset(*test_data)

    del tokenizer
    torch.cuda.empty_cache()

    if method == 'class-weight':
        class_weights = compute_class_weights([item['labels'] for item in train_dataset])
    else:
        class_weights = None

    return train_dataset, val_dataset, test_dataset, null_count, class_weights

In [7]:
def train(method, absa_method, model_name):
    f1_macro_aspect = 0
    precision_macro_aspect = 0
    recall_macro_aspect = 0
    TFs = (0, 0, 0, 0)

    # gom labels/preds toàn bộ aspects để tính macro-class
    all_labels, all_preds = [], []

    for aspect in aspects:
        print(f'------ Aspect: {aspect}------')
        train_dataset, val_dataset, test_dataset, val_null_count, class_weights = get_datataset_and_model(method, absa_method, aspect)
        config = BERTConfig(absa_method, num_classes=3, class_weights=class_weights)
        model = BERTModel(config).to(device)

        num_epochs=25

        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)

        batch_size=64
        lr = 2e-5
        weight_decay = 0.01

        result = train_and_eval_model(
            aspect, method, absa_method, model,
            train_dataset, val_dataset, test_dataset,
            val_null_count,
            batch_size=batch_size, num_epochs=num_epochs,
            lr=lr, weight_decay=weight_decay, patience=5
        )

        # cộng dồn macro-F1 per aspect
        f1_macro_aspect += result['eval_f1_macro']
        precision_macro_aspect += result['eval_precision_macro']
        recall_macro_aspect += result['eval_recall_macro']
        TFs = np.add(TFs, result['eval_total_tf']).tolist()

        # gom labels/preds để tính macro-class flatten
        all_labels.extend(result["eval_labels"])
        all_preds.extend(result["eval_preds"])

        del model, train_dataset, val_dataset
        torch.cuda.empty_cache()

    # --- Macro theo aspect (average đều các aspect) ---
    num_aspects = len(aspects)
    f1_macro_aspect /= num_aspects
    precision_macro_aspect /= num_aspects
    recall_macro_aspect /= num_aspects

    # --- Macro theo class (flatten all aspects) ---
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:\n", cm)
    from sklearn.metrics import precision_recall_fscore_support
    p_macro_class, r_macro_class, f1_macro_class, _ = precision_recall_fscore_support(
        all_labels, all_preds, average="macro", zero_division=0
    )

    # --- Micro (global TP/FP/FN) ---
    total_tp, total_fp, total_tn, total_fn = TFs
    precision_micro = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall_micro = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1_micro = (2 * precision_micro * recall_micro) / (precision_micro + recall_micro) if (precision_micro + recall_micro) > 0 else 0

    torch.cuda.empty_cache()

    return {
        # macro-aspect
        "f1_macro_avg": f1_macro_aspect,
        "precision_macro_avg": precision_macro_aspect,
        "recall_macro_avg": recall_macro_aspect,
        # macro-class (flatten)
        "f1_macro_class": f1_macro_class,
        "precision_macro_class": p_macro_class,
        "recall_macro_class": r_macro_class,
        # micro
        "f1_micro": f1_micro,
        "precision_micro": precision_micro,
        "recall_micro": recall_micro,
    }


In [8]:
for model in ["BERT"]:
    print(f'------ Model: {model}------')
    for (method, absa_method) in [
                                  ('class-weight', None), 
                                  ('class-weight', 'min'), 
                                  ('class-weight', 'max'), 
                                  ('class-weight', 'avg'),
                                  ('upsample_replacement', None),
                                  ('upsample_replacement', 'min'),
                                  ('upsample_replacement', 'max'),
                                  ('upsample_replacement', 'avg')
                                 ]:
        result = train(method, absa_method, model)

        if absa_method is None:
            absa_method_str = "None"
        else:
            absa_method_str = absa_method

        eval_metrics = {
            'method': method,
            'absa_method': absa_method_str,

            # --- macro ---
            'f1_macro_avg': result['f1_macro_avg'],
            'precision_macro_avg': result['precision_macro_avg'],
            'recall_macro_avg': result['recall_macro_avg'],

            'f1_macro_class': result['f1_macro_class'],
            'precision_macro_class': result['precision_macro_class'],
            'recall_macro_class': result['recall_macro_class'],

            # --- micro ---
            'f1_micro': result['f1_micro'],
            'precision_micro': result['precision_micro'],
            'recall_micro': result['recall_micro']
        }
        
        print("FINAL: ", eval_metrics)
        
        torch.cuda.empty_cache()


------ Model: BERT------
------ Aspect: appearance------


README.md:   0%|          | 0.00/923 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/211k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/32.4k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/33.0k [00:00<?, ?B/s]

...

Test results: {'eval_loss': 1.676090955734253, 'eval_model_preparation_time': 0.0054, 'eval_accuracy': 0.5414364640883977, 'eval_f1_macro': 0.5171527424975252, 'eval_precision_macro': 0.5571981497215143, 'eval_recall_macro': 0.5128330084470435, 'eval_f1_micro': 0.5414364640883977, 'eval_precision_micro': 0.5414364640883977, 'eval_recall_micro': 0.5414364640883977, 'eval_total_tf': [98, 83, 279, 83], 'eval_runtime': 0.6765, 'eval_samples_per_second': 267.539, 'eval_steps_per_second': 4.434}
------ Aspect: taste------


100%|██████████| 375/375 [00:00<00:00, 776.19it/s]


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro,F1 Micro,Precision Micro,Recall Micro,Total Tf
1,1.118800,1.076449,0.421918,0.337270,0.296051,0.415127,0.421918,0.421918,0.421918,"[154, 211, 519, 211]"
2,0.921900,0.960958,0.542466,0.538495,0.537643,0.539562,0.542466,0.542466,0.542466,"[198, 167, 563, 167]"
3,0.745800,0.968624,0.531507,0.529166,0.534040,0.528200,0.531507,0.531507,0.531507,"[194, 171, 559, 171]"
4,0.520600,1.064699,0.558904,0.555503,0.556317,0.555692,0.558904,0.558904,0.558904,"[204, 161, 569, 161]"
5,0.368600,1.145083,0.550685,0.544548,0.544281,0.546912,0.550685,0.550685,0.550685,"[201, 164, 566, 164]"
6,0.319100,1.265512,0.553425,0.554075,0.559032,0.550548,0.553425,0.553425,0.553425,"[202, 163, 567, 163]"
7,0.227000,1.389112,0.547945,0.548221,0.552902,0.545547,0.547945,0.547945,0.547945,"[200, 165, 565, 165]"
8,0.172400,1.491604,0.556164,0.549924,0.548934,0.552493,0.556164,0.556164,0.556164,"[203, 162, 568, 162]"
9,0.144100,1.574980,0.536986,0.536557,0.542261,0.534905,0.536986,0.536986,0.536986,"[196, 169, 561, 169]"


Uploading...:   0%|          | 0.00/876M [00:00<?, ?B/s]

Test results: {'eval_loss': 0.9725877046585083, 'eval_model_preparation_time': 0.0053, 'eval_accuracy': 0.5813333333333334, 'eval_f1_macro': 0.5878543141909317, 'eval_precision_macro': 0.5832074748967141, 'eval_recall_macro': 0.5948791448791448, 'eval_f1_micro': 0.5813333333333334, 'eval_precision_micro': 0.5813333333333334, 'eval_recall_micro': 0.5813333333333334, 'eval_total_tf': [218, 157, 593, 157], 'eval_runtime': 1.3914, 'eval_samples_per_second': 269.52, 'eval_steps_per_second': 4.312}
Confusion Matrix:
 [[180 102  16]
 [ 87 259  92]
 [ 17 111 198]]
FINAL:  {'method': 'upsample_replacement', 'absa_method': 'avg', 'f1_macro_avg': 0.5950037571394823, 'precision_macro_avg': 0.6126294788791047, 'recall_macro_avg': 0.5948484723202512, 'f1_macro_class': 0.60478991624757, 'precision_macro_class': 0.6098634846633808, 'recall_macro_class': 0.6009043365803367, 'f1_micro': 0.5998116760828626, 'precision_micro': 0.5998116760828626, 'recall_micro': 0.5998116760828626}
